In [ ]:
%env DASHSCOPE_API_KEY=请到千问灵积官方（https://dashscope.console.aliyun.com/apiKey）申请免费Key，并替换为自己的
%env TAVILY_API_KEY=请到Tavily官方（https://app.tavily.com/）申请免费Key，并替换为自己的
%env AMAP_API_KEY=请到高德开发者中心（https://console.amap.com/dev/index），申请免费key，并替换为自己的

In [2]:
!pip install -qU dashscope

In [3]:
import langchain, langchain_core, langchain_community, dashscope

for module in (langchain, langchain_core, langchain_community, dashscope):
    if hasattr(module, '__version__'):
        print(f"{module.__name__:<20}\t{module.__version__}")

langchain           	0.2.3
langchain_core      	0.2.5
langchain_community 	0.2.1


In [4]:
!pip list | grep -i dashscope

dashscope                                1.19.2


# 绑定工具

In [5]:
import os
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.tools import tool

使用通义千问官方API

In [6]:
model = ChatTongyi(model='qwen-turbo')

工具定义及工具绑定

## 方法一：自定义工具声明

In [7]:
def search_web(query, k=5):
    import os
    from langchain_community.retrievers import TavilySearchAPIRetriever
    
    retriever = TavilySearchAPIRetriever(k=5)
    documents = retriever.invoke(query)
    # return [{'title': doc.metadata['title'], 'abstract': doc.page_content, 'href': doc.metadata['source'], 'score': doc.metadata['score']} for doc in
    #         documents]
    content = '\n\n'.join([doc.page_content for doc in documents])
    prompt = f"""请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
{content}
<<</content>>>
"""
    print('prompt:')
    print(prompt)
    
    return model.invoke(prompt).content
    

def get_current_weather(city):
    import requests
    from datetime import datetime
    import os
    
    amap_api_key = os.environ['AMAP_API_KEY']
    url = f'https://restapi.amap.com/v3/config/district?keywords={city}&key={amap_api_key}'
    resp = requests.get(url)
    # print('行政区划：')
    # print(resp.json())
    
    adcode = resp.json()['districts'][0]['adcode']
    # adcode = '110000'

    url = f'https://restapi.amap.com/v3/weather/weatherInfo?city={adcode}&key={amap_api_key}&extensions=base'
    resp = requests.get(url)
    """
    {'province': '北京',
     'city': '北京市',
     'adcode': '110000',
     'weather': '晴',
     'temperature': '26',
     'winddirection': '西北',
     'windpower': '4',
     'humidity': '20',
     'reporttime': '2024-05-26 13:38:38',
     'temperature_float': '26.0',
     'humidity_float': '20.0'}
    """
    # print('天气：')
    # print(resp.json())
    weather_json = resp.json()['lives'][0]
    return f"{weather_json['city']}{datetime.strptime(weather_json['reporttime'], '%Y-%m-%d %H:%M:%S').strftime('%m月%d日')}{weather_json['weather']}，气温{weather_json['temperature']}摄氏度，{weather_json['winddirection']}风{weather_json['windpower']}级"
    
    
fn_map = {
    'get_current_weather': get_current_weather,
    'search_web': search_web
}
tools = [
    {
        "name": "get_current_weather",
        "description": "根据城市名获取天气",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "城市名，例如北京"
                }
            },
            "required": ["city"]
        }
    },
    {
        "name": "search_web",
        "description": "当有不知道或不确定的信息时，搜索互联网",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "要搜索的内容"
                }
            },
            "required": ["query"]
        }
    }
]


llm_with_tool = model.bind_tools(
    tools=tools
)

## 方法二：使用tool装饰器

注意，使用这种方式，方法的docstring是必须的，否则在执行时会报错，原因也很简单，在向LLM对话时，LLM就是靠docstring信息来感知工具作用的

In [8]:
from langchain_core.tools import tool
    
@tool
def search_web2(query: str, k=5) -> str:
    """当有不知道或不确定的信息时，搜索互联网"""
    
    import os
    from langchain_community.retrievers import TavilySearchAPIRetriever
    
    retriever = TavilySearchAPIRetriever(k=5)
    documents = retriever.invoke(query)
    # return [{'title': doc.metadata['title'], 'abstract': doc.page_content, 'href': doc.metadata['source'], 'score': doc.metadata['score']} for doc in
    #         documents]
    content = '\n\n'.join([doc.page_content for doc in documents])
    prompt = f"""请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
{content}
<<</content>>>
"""
    print('prompt:')
    print(prompt)
    
    return model.invoke(prompt).content
    
@tool
def get_current_weather2(city: str) -> str:
    """根据城市名获取天气"""
    
    import requests
    from datetime import datetime
    import os
    
    amap_api_key = os.environ['AMAP_API_KEY']
    url = f'https://restapi.amap.com/v3/config/district?keywords={city}&key={amap_api_key}'
    resp = requests.get(url)
    # print('行政区划：')
    # print(resp.json())
    
    adcode = resp.json()['districts'][0]['adcode']
    # adcode = '110000'

    url = f'https://restapi.amap.com/v3/weather/weatherInfo?city={adcode}&key={amap_api_key}&extensions=base'
    resp = requests.get(url)
    """
    {'province': '北京',
     'city': '北京市',
     'adcode': '110000',
     'weather': '晴',
     'temperature': '26',
     'winddirection': '西北',
     'windpower': '4',
     'humidity': '20',
     'reporttime': '2024-05-26 13:38:38',
     'temperature_float': '26.0',
     'humidity_float': '20.0'}
    """
    # print('天气：')
    # print(resp.json())
    weather_json = resp.json()['lives'][0]
    return f"{weather_json['city']}{datetime.strptime(weather_json['reporttime'], '%Y-%m-%d %H:%M:%S').strftime('%m月%d日')}{weather_json['weather']}，气温{weather_json['temperature']}摄氏度，{weather_json['winddirection']}风{weather_json['windpower']}级"
    
    
fn_map2 = {
    'get_current_weather2': get_current_weather2,
    'search_web2': search_web2
}

llm_with_tool2 = model.bind_tools(tools=[get_current_weather2, search_web2])

# 使用工具

## 天气查询

### 方法一

In [9]:
from langchain_core.messages import HumanMessage
import json

ai_msg = llm_with_tool.invoke("北京今天的天气怎么样")
fn_name = ai_msg.additional_kwargs['tool_calls'][0]['function']['name']
kwargs = json.loads(ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments'])

In [10]:
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"city": "北京"}'}, 'id': '', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '85cc4446-d14d-970d-8755-362838b71fc4', 'token_usage': {'input_tokens': 254, 'output_tokens': 18, 'total_tokens': 272}}, id='run-70471ca2-2eef-493e-b28a-92de0c7a7648-0', tool_calls=[{'name': 'get_current_weather', 'args': {'city': '北京'}, 'id': ''}])

从结果看，触发函数调用了

In [11]:
ai_msg.additional_kwargs

{'tool_calls': [{'function': {'name': 'get_current_weather',
    'arguments': '{"city": "北京"}'},
   'id': '',
   'type': 'function'}]}

In [12]:
fn_map[fn_name](**kwargs)

'北京市06月09日小雨，气温27摄氏度，南风≤3级'

#### llm_with_tool.invoke实际发送的请求

llm_with_tool.invoke("北京今天的天气怎么样")这一句，实际在底层发送请求时，调用的是dashscope库中，dashscope/api_entities/http_request.py这个文件的_handle_request方法

发送的是一个HTTP POST方法，具体内容如下：

```json
{
    "model": "qwen-turbo",
    "parameters": {
        "top_p": 0.8,
        "result_format": "message",
        "tools": [
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "根据城市名获取天气",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "城市名，例如北京"
                            }
                        },
                        "required": [
                            "city"
                        ]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_web",
                    "description": "当有不知道或不确定的信息时，搜索互联网",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "要搜素的内容"
                            }
                        },
                        "required": [
                            "query"
                        ]
                    }
                }
            }
        ]
    },
    "input": {
        "messages": [
            {
                "role": "user",
                "content": "北京今天的天气怎么样"
            }
        ]
    }
}
```

### 方法二

In [31]:
ai_msg = llm_with_tool2.invoke("北京今天的天气怎么样")
fn_name = ai_msg.additional_kwargs['tool_calls'][0]['function']['name']
kwargs = json.loads(ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments'])

注意，由于使用了`@tool`包装器，所以，不能直接使用`**kwargs`的方式传参了，否则会报错，需要显式地把参数取出来

In [14]:
fn_map2[fn_name](kwargs['city'])

/Library/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'北京市06月09日小雨，气温27摄氏度，南风≤3级'

使用方法二，其实Langchain官方更推荐使用`.invoke`的方式进行调用

In [33]:
fn_map2[fn_name].invoke(kwargs['city'])

'北京市06月09日小雨，气温27摄氏度，南风≤3级'

## 网络搜索

In [15]:
ai_msg = llm_with_tool.invoke("庆余年2好看吗")

In [16]:
ai_msg

AIMessage(content='"庆余年2"目前还没有正式播出或者上映，因为它是一部基于小说改编的电视剧，第一季在2019年就已经完结并受到了观众的喜爱。通常续集的质量会受到原作粉丝和新观众的期待，但是否好看因人而异，取决于个人喜好和对剧情发展的看法。\n\n如果你喜欢第一部的剧情和演员表现，可能会对第二季有所期待。不过，具体的评价和观看体验需要等到官方发布或者播出后，观众们看过并给出反馈后才能得出结论。如果你想了解关于庆余年系列的最新动态，我可以帮你搜索相关资讯。', response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '902feb76-9fe5-9fb4-90d9-f07f549d1eed', 'token_usage': {'input_tokens': 256, 'output_tokens': 127, 'total_tokens': 383}}, id='run-f221755d-7305-417a-9194-515ddd9e9e08-0')

从结果看，并没有触发函数调用

In [17]:
ai_msg = llm_with_tool.invoke("请帮我搜索并回答庆余年2好看吗")

In [18]:
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'search_web', 'arguments': '{"query": "庆余年2 好看吗"}'}, 'id': '', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '76a7f075-62f2-9e37-9628-e69ce7f3586e', 'token_usage': {'input_tokens': 262, 'output_tokens': 25, 'total_tokens': 287}}, id='run-14b8fc06-1665-4d39-9ec0-a3d0b6fccc03-0', tool_calls=[{'name': 'search_web', 'args': {'query': '庆余年2 好看吗'}, 'id': ''}])

在query中显式指定需要“搜索”，成功触发函数调用

In [19]:
fn_name = ai_msg.additional_kwargs['tool_calls'][0]['function']['name']
kwargs = json.loads(ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments'])

In [20]:
fn_name

'search_web'

In [21]:
kwargs

{'query': '庆余年2 好看吗'}

从query参数来看，LLM对输入做了分词，并将其转换成了适合搜索的关键词形式

调用函数

In [26]:
fn_map[fn_name](**kwargs)

prompt:
请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
这篇剧评可能有剧透. 《庆余年第二季》首播不负所望，台网双爆，热度空前，"年度剧王"的桂冠看来差不多已经戴稳了。. 多数观众认为第二季没有拉胯，延续了第一季的精彩，尤其是抱月楼众皇子与范闲上演狂飙大乱斗这场戏，着实精彩好看，堪称该剧的第 ...

央视热播剧《庆余年2》近日迎来了大结局。 从播出到现在，《庆余年2》频繁登上热搜。 关于它的角色、剧情，一直讨论不断。 电视剧虽然只有几十集，却囊括了朝堂权谋，江湖纷争、市井生活与人情百态。 看的是剧，讲的是故事，品的却是人生。

观众的打分和评价都一针见血_腾讯新闻. 《庆余年2》第一波真实口碑出炉!. 观众的打分和评价都一针见血. 等了5年，终于等到《 庆余年2 》惊喜开播了。. 而这部剧的首播当晚排面有多大呢？. 全网热度第一，热搜前十八个都和这部剧有关系。. 这个架势，不愧 ...

庆余年2前5集观后感，优缺点浅评. 1～2集：前两集不太好看，因为看庆2之前我刚把第一季精编版刷完重温了一下剧情，感觉节奏和味道都不太对，没有庆1的节奏感，剧情很平，梗多，但是信息量很低，有些水，假死剧情逻辑上来说圆得很生硬，没什么必要 ...

庆余年2开播这天，我给我自己放了一首《好日子》。 因为点开第一集刚看了个开头，我就知道，属于我的2019年重新杀了回来。 故事的接续感诚意十足，一点开就是张庆被催更，回扣整体的世界观，再跟随故事回到剧中，以同样的节奏同样的视角重现第一季的 ...
<<</content>>>



'《庆余年第二季》首播受到热烈欢迎，被誉为“年度剧王”，凭借精彩的剧情和延续的第一季风格赢得了观众的认可，尤其抱月楼皇子与范闲的戏份备受称赞。剧集涵盖了多种元素如权谋、江湖、生活和人性探索，引发大量讨论。观众评分积极，首播当晚热度极高，与第一部作品有强烈的连结感。然而，前几集中部分观众反映与第一部的节奏相比稍显平淡，存在一些逻辑上的生硬之处。总体来看，该剧在观众中引起了强烈反响。'

### 方法二

In [35]:
ai_msg = llm_with_tool2.invoke("请帮我搜索并回答庆余年2好看吗")
fn_name = ai_msg.additional_kwargs['tool_calls'][0]['function']['name']
kwargs = json.loads(ai_msg.additional_kwargs['tool_calls'][0]['function']['arguments'])

In [37]:
fn_map2[fn_name].invoke(kwargs['query'])

prompt:
请将下面这段内容（<<<content>>><<</content>>>包裹的部分）进行总结：
<<<content>>>
作为备受期待的续集，《庆余年2》承载了大量观众的期望。然而，这部作品在多方面未能达到前作的高度。本文将从逻辑漏洞、叙事节奏、角色塑造以及制作质量等角度进行全面的批评性分析。 逻辑漏洞与情节问题 《庆余年2》在情节发展上充满了明显的逻辑 ...

影评; 2023年度榜单 ... 庆余年2开播这天，我给我自己放了一首《好日子》。 因为点开第一集刚看了个开头，我就知道，属于我的2019年重新杀了回来。 故事的接续感诚意十足，一点开就是张庆被催更，回扣整体的世界观，再跟随故事回到剧中，以同样的节奏同样 ...

1～2集：前两集不太好看，因为看庆2之前我刚把第一季精编版刷完重温了一下剧情，感觉节奏和味道都不太对，没有庆1的节奏感，剧情很平，梗多，但是信息量很低，有些水，假死剧情逻辑上来说圆得很生硬，没什么必要，去掉不影响，而且第一集前半段现代剧情太水了，什么人都去催更那里有 ...

由于第二季的体量仅有36集，因此很难不让人以为《庆余年2》的节奏很快，一定会单刀直入的开始讲第二季的情节。. 但编剧和导演显然不知道观众们想看的是什么，第一集就让人看得想破口大骂。. 随着首播口碑出炉，看看观众们的评论和打分就知道，一针见血 ...

观众的打分和评价都一针见血_腾讯新闻. 《庆余年2》第一波真实口碑出炉!. 观众的打分和评价都一针见血. 等了5年，终于等到《 庆余年2 》惊喜开播了。. 而这部剧的首播当晚排面有多大呢？. 全网热度第一，热搜前十八个都和这部剧有关系。. 这个架势，不愧 ...
<<</content>>>



'《庆余年2》作为续集，未能满足观众的期望，受到批评，主要问题包括逻辑漏洞、叙事节奏不协调、角色发展不足和制作质量问题。前两集被指出与前作差距较大，节奏感减弱，剧情显得平庸且信息量低，部分情节处理生硬。观众反应强烈，首播口碑不佳，打分和评价直接反映出对剧集质量的不满。尽管开播时场面火爆，但实际内容并未达到预期。'